In [7]:
# Cell 1: Install (run once; skip if already installed)
!pip install opencv-python-headless==4.7.0.72 face_recognition numpy pandas matplotlib ipywidgets pillow fpdf2 twilio
# Note: If dlib / face_recognition fails on your system, you might need to install system packages:
# On Ubuntu: sudo apt-get install build-essential cmake libopenblas-dev liblapack-dev
# On Windows: install Visual Studio Build Tools + CMake, or use prebuilt dlib wheel.


     ---------------------------------------- 0.0/100.1 MB ? eta -:--:--
     - ------------------------------------- 3.4/100.1 MB 17.3 MB/s eta 0:00:06
     -- ------------------------------------ 7.1/100.1 MB 17.4 MB/s eta 0:00:06
     ---- --------------------------------- 10.7/100.1 MB 17.1 MB/s eta 0:00:06
     ----- -------------------------------- 13.6/100.1 MB 16.1 MB/s eta 0:00:06
     ------ ------------------------------- 16.5/100.1 MB 15.6 MB/s eta 0:00:06
     ------- ------------------------------ 19.1/100.1 MB 15.0 MB/s eta 0:00:06
     -------- ----------------------------- 22.0/100.1 MB 14.7 MB/s eta 0:00:06
     --------- ---------------------------- 25.2/100.1 MB 14.8 MB/s eta 0:00:06
     ---------- --------------------------- 26.7/100.1 MB 14.4 MB/s eta 0:00:06
     ----------- -------------------------- 29.9/100.1 MB 13.9 MB/s eta 0:00:06
     ------------ ------------------------- 33.0/100.1 MB 14.1 MB/s eta 0:00:05
     ------------- ------------------------ 36.

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [41 lines of output]
  running bdist_wheel
  running build
  running build_ext
  
  
                     CMake is not installed on your system!
  
      Or it is possible some broken copy of cmake is installed on your system.
      It is unfortunately very common for python package managers to include
      broken copies of cmake.  So if the error above this refers to some file
      path to a cmake file inside a python or anaconda or miniconda path then you
      should delete that broken copy of cmake from your computer.
  
      Instead, please get an official copy of cmake from one of these known good
      sources of an official cmake:
          - cmake.org (this is how windows users should get cmake)
          - apt install cmake (for Ubuntu or Debian based systems)
          - yum install cmake (for Redhat or CenOS based systems)
  
      On a linux machine you can run `which cmake` to see what cmake you are
      act

In [8]:
# Cell 2: Imports & config
import os, io, time, threading, pickle
from datetime import datetime
import numpy as np
import pandas as pd
import cv2
import face_recognition
from IPython.display import display, clear_output, Image
import ipywidgets as widgets
import matplotlib.pyplot as plt
from fpdf import FPDF

# Files
ENC_PATH = "encodings.pkl"
ATT_PATH = "attendance.csv"
CAPTURE_IMAGES = 5
TOLERANCE = 0.48
CAM_INDEX = 0

# Thread control
_camera_thread = None
_camera_running = False


ModuleNotFoundError: No module named 'face_recognition'

In [ ]:
# Cell 3: Helpers
def load_encodings():
    if os.path.exists(ENC_PATH):
        try:
            return pickle.load(open(ENC_PATH, "rb"))
        except:
            return {"names": [], "encodings": []}
    return {"names": [], "encodings": []}

def save_encodings(data):
    pickle.dump(data, open(ENC_PATH, "wb"))

def ensure_attendance_file():
    if not os.path.exists(ATT_PATH):
        pd.DataFrame(columns=["name","timestamp"]).to_csv(ATT_PATH, index=False)

def log_attendance(name):
    ensure_attendance_file()
    df = pd.read_csv(ATT_PATH)
    ts = datetime.now().isoformat(sep=' ', timespec='seconds')
    df = df.append({"name": name, "timestamp": ts}, ignore_index=True)
    df.to_csv(ATT_PATH, index=False)

def export_attendance_pdf(pdf_path="attendance_report.pdf"):
    if not os.path.exists(ATT_PATH):
        return False
    df = pd.read_csv(ATT_PATH)
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Attendance Report", ln=True, align="C")
    pdf.ln(5)
    for i, row in df.iterrows():
        pdf.cell(0, 8, txt=f"{row['timestamp']}  -  {row['name']}", ln=True)
    pdf.output(pdf_path)
    return True


In [ ]:
# Cell 4: Anti-spoofing heuristic
def is_spoof(frame):
    """
    Lightweight heuristic: measure Laplacian variance (blurriness/texture).
    Lower variance often indicates printed/photo/screen; tweak threshold per camera.
    Returns True if likely spoof, False if likely live.
    """
    try:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        lap = cv2.Laplacian(gray, cv2.CV_64F)
        var = lap.var()
        # threshold: tune if needed (70 is a starting point)
        return var < 70
    except Exception as e:
        return False


In [ ]:
# Cell 5: Register UI function (callable)
def register_person_interactive(name, camera_index=CAM_INDEX):
    if not name.strip():
        print("Provide a valid name")
        return False
    cap = cv2.VideoCapture(camera_index)
    if not cap.isOpened():
        print("Cannot open webcam. Check camera index.")
        return False

    print(f"Registering: {name} — look at the camera")
    encs = []
    collected = 0
    while collected < CAPTURE_IMAGES:
        ret, frame = cap.read()
        if not ret:
            print("Frame read failed, retrying...")
            time.sleep(0.2)
            continue
        rgb = frame[:, :, ::-1]
        boxes = face_recognition.face_locations(rgb)
        if boxes:
            e = face_recognition.face_encodings(rgb, boxes)[0]
            encs.append(e)
            collected += 1
            print(f"Captured {collected}/{CAPTURE_IMAGES}")
        # show small window for feedback
        cv2.imshow("Register - press q to abort", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    if not encs:
        print("No face captured. Registration failed.")
        return False

    mean_enc = np.mean(encs, axis=0)
    data = load_encodings()
    data["names"].append(name)
    data["encodings"].append(mean_enc)
    save_encodings(data)
    print(f"Registered {name} successfully.")
    return True


In [9]:
# Cell 6: Camera loop + notebook live preview
from IPython.display import display, clear_output

def _camera_loop(output_widget, camera_index=CAM_INDEX):
    global _camera_running
    data = load_encodings()
    known_encs = data["encodings"]
    known_names = data["names"]
    cap = cv2.VideoCapture(camera_index)
    if not cap.isOpened():
        with output_widget:
            print("Cannot open webcam. Stop and check camera.")
        _camera_running = False
        return

    last_seen = {}  # cooldown per person
    COOLDOWN = 20  # seconds
    _camera_running = True

    try:
        while _camera_running:
            ret, frame = cap.read()
            if not ret:
                time.sleep(0.1)
                continue

            display_frame = frame.copy()
            # quick spoof check on whole frame
            spoof = is_spoof(frame)
            rgb = frame[:, :, ::-1]
            boxes = face_recognition.face_locations(rgb)
            encodings = face_recognition.face_encodings(rgb, boxes)

            for (top, right, bottom, left), enc in zip(boxes, encodings):
                # scale coords already in same frame size
                name = "Unknown"
                if known_encs:
                    dists = face_recognition.face_distance(known_encs, enc)
                    best = np.argmin(dists)
                    if dists[best] < TOLERANCE:
                        name = known_names[best]

                # draw rectangle & label
                color = (0,255,0) if name!="Unknown" else (0,0,255)
                cv2.rectangle(display_frame, (left, top), (right, bottom), color, 2)
                cv2.putText(display_frame, name, (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

                # logging/confidence
                if name!="Unknown" and not spoof:
                    now = time.time()
                    last = last_seen.get(name, 0)
                    if now - last > COOLDOWN:
                        log_attendance(name)
                        last_seen[name] = now
                elif name=="Unknown":
                    # optional: handle unknown (could log image, send alert)
                    pass

            # overlay spoof warning
            if spoof:
                cv2.putText(display_frame, "SPOOF SUSPECTED", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 3)

            # show image in output widget
            _, img_encoded = cv2.imencode('.jpg', display_frame)
            img_bytes = img_encoded.tobytes()
            with output_widget:
                clear_output(wait=True)
                display(Image(data=img_bytes))
            time.sleep(0.03)
    except Exception as e:
        with output_widget:
            print("Camera loop error:", e)
    finally:
        cap.release()
        _camera_running = False

def start_camera(output_widget, camera_index=CAM_INDEX):
    global _camera_thread, _camera_running
    if _camera_running:
        with output_widget:
            print("Camera already running")
        return
    _camera_thread = threading.Thread(target=_camera_loop, args=(output_widget, camera_index), daemon=True)
    _camera_thread.start()

def stop_camera():
    global _camera_running, _camera_thread
    _camera_running = False
    # give thread time to stop
    time.sleep(0.5)
    _camera_thread = None


NameError: name 'CAM_INDEX' is not defined

In [10]:
# Cell 7: Build GUI widgets and display them
ensure_attendance_file()

# Widgets
out_camera = widgets.Output()
name_text = widgets.Text(description="Name:", placeholder="Full name")
btn_register = widgets.Button(description="Register", button_style="success")
btn_start = widgets.Button(description="Start", button_style="primary")
btn_stop = widgets.Button(description="Stop", button_style="warning")
btn_refresh = widgets.Button(description="Refresh Logs")
btn_export_csv = widgets.Button(description="Export CSV")
btn_export_pdf = widgets.Button(description="Export PDF")
btn_clear = widgets.Button(description="Clear Logs", button_style="danger")

# Handlers
def on_register(b):
    nm = name_text.value.strip()
    if not nm:
        print("Enter name first.")
        return
    print("Registration started. Webcam will open. Press q in the window to abort.")
    ok = register_person_interactive(nm)
    if ok:
        print("Registration done.")
    else:
        print("Registration failed / aborted.")

def on_start(b):
    print("Starting camera...")
    start_camera(out_camera, CAM_INDEX)

def on_stop(b):
    print("Stopping camera...")
    stop_camera()

def on_refresh(b):
    clear_output(wait=True)
    display(ui)
    display_dashboard_contents()

def on_export_csv(b):
    if os.path.exists(ATT_PATH):
        from IPython.display import FileLink
        print("CSV exported:", ATT_PATH)
        display(FileLink(ATT_PATH))
    else:
        print("No attendance file to export.")

def on_export_pdf(b):
    ok = export_attendance_pdf()
    if ok:
        from IPython.display import FileLink
        print("PDF exported: attendance_report.pdf")
        display(FileLink("attendance_report.pdf"))
    else:
        print("No attendance to export.")

def on_clear(b):
    pd.DataFrame(columns=["name","timestamp"]).to_csv(ATT_PATH, index=False)
    print("Attendance cleared.")

btn_register.on_click(on_register)
btn_start.on_click(on_start)
btn_stop.on_click(on_stop)
btn_refresh.on_click(on_refresh)
btn_export_csv.on_click(on_export_csv)
btn_export_pdf.on_click(on_export_pdf)
btn_clear.on_click(on_clear)

controls_row = widgets.HBox([name_text, btn_register, btn_start, btn_stop, btn_refresh])
controls_row2 = widgets.HBox([btn_export_csv, btn_export_pdf, btn_clear])

ui = widgets.VBox([controls_row, controls_row2, out_camera])

display(ui)


NameError: name 'ensure_attendance_file' is not defined

In [11]:
# Cell 8: Dashboard contents (call display_dashboard_contents())
def display_dashboard_contents():
    ensure_attendance_file()
    df = pd.read_csv(ATT_PATH)
    print("\n📜 Recent Attendance (last 20):")
    if len(df)==0:
        print("No records yet.")
    else:
        display(df.tail(20))

    # plot daily counts
    if len(df)>0:
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df['date'] = df['timestamp'].dt.date
        daily = df.groupby('date').count()['name']
        plt.figure(figsize=(7,3))
        daily.plot(kind='bar')
        plt.title("Daily Attendance Count")
        plt.xlabel("Date")
        plt.ylabel("Count")
        plt.show()

# show once initially
display_dashboard_contents()


NameError: name 'ensure_attendance_file' is not defined